## Pipeline

Tot nu toe is de code voor het opbouwen van een model vespreid over twee stappen. 
Een preprocesing stap waar alle transformaties op gebeuren op de data bruikbaar te maken zoals schaling, invullen van ontbrekende waarden, hogere orde features, ...
In een latere stap gebeurd dan het trainen van een model.
Echter is het zo dat als je het model in gebruik wilt nemen in een applicatie dat je ook exact dezelfde preprocessing stappen nodig hebt omdat je de data die je aan het model presenteert hetzelfde moet zijn.
Om deze reden is de vorige aanpak niet praktisch aangezien het eenvoudig is om een fout te maken.
Het is ook geen onderhoudbare oplossing omdat het veel manueel werk vereist indien er een bepaalde stap aangepakt wordt.

Om dit tegen te gaan kan men gebruik maken van Pipelines. 
Hiermee definieer je de flow die de data moet ondergaan om verwerkt te worden. 
De volledige pipeline of flow kan dan gebruikt worden om te trainen en te gebruiken in productie. 
Hierdoor worden de preprocessing stappen (zoals scalers, ordinal of one-hotencoder) steeds enkel op de training folds uitgevoerd.
Een bijkomende voordeel is dat ook parameters van de preprocessing stappen meegenomen kunnen worden in de gridsearch.

Let op dat NaN waarden invullen kan gebeuren in de pipeline maar rijen weglaten kan niet gedaan worden tijdens de pipeline. Echter is het gebruik van een pipeline een goede stap in het verhogen van de leesbaarheid van een code stuk en wordt het daardoor veelvuldig gebruikt.

In [ ]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

np.random.seed(0)

X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
X

In [12]:
categorical_features = ['sex', 'embarked', 'pclass', 'parch'] # hier kunnen er eventueel nog een aantal bij
numerical_features = ['age', 'sibsp', 'fare']

numeric_transformer = Pipeline(steps=[
    ('imp', SimpleImputer()), # simple imputer is om null-waardne in te vullen
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imp', SimpleImputer(missing_values='unknown', strategy='constant')),
    ('enc', OneHotEncoder())
])

preprocessor = ColumnTransformer(transformers=[
    # bij een columntransformer hebben we hier een tuple met drie waarden (key, een pipeline/transformer, een lijst met kolommen)
    #('num', numeric_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
])

clf = Pipeline(steps=[
    # bij een pipeline heb je hier een tuple met twee waarden (key, een pipeline of stap)
    ('preprocessor', preprocessor),
    ('clf', LogisticRegression())
])

In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'clf__C': [10,100,1000],
    'preprocessor__cat__imp__missing_values': ['1'] # let op de dubbel underscore bij het gebruiken van de keys
}

gridsearch = GridSearchCV(clf, param_grid, cv=5)

gridsearch.fit(X,y)

/usr/local/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:953: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 942, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/usr/local/lib/python3.13/site-packages/sklearn/metrics/_scorer.py", line 492, in __call__
    return estimator.score(*args, **kwargs)
           ~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.13/site-packages/sklearn/pipeline.py", line 1185, in score
    Xt = transform.transform(Xt)
  File "/usr/local/lib/python3.13/site-packages/sklearn/utils/_set_output.py", line 316, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "/usr/local/lib/python3.13/site-packages/sklearn/compose/_column_transformer.py", line 1096, in transform
    Xs = self._call_

ValueError: Input contains NaN

In [ ]:
preprocessor.fit_transform(X)

## Oefening

Maak een pipeline voor een gridsearch uit te voeren op de diabetes-dataset in sklearn. Gebruik een Min-Max scaler voor de numerieke waarden en een ordinal encoder voor de categorieke kolommen.
Zoek naar de beste combinatie van hyperparameters van een SVM-classifier door middel van een grid search met cross validation.